In [26]:
import numpy as np
from scipy import stats

In [27]:
aspirin_data = np.array( [1 if i<104 else 0 for i in range(11037)] )
placebo_data = np.array( [1 if i<189 else 0 for i in range(11034)] )

In [28]:
aspirin_data

array([1, 1, 1, ..., 0, 0, 0])

In [29]:
prob_infarction_aspirin = aspirin_data.sum() / aspirin_data.shape[0]
prob_infarction_placebo = placebo_data.sum() / placebo_data.shape[0]
print('Infarction probability (aspirin): %.4f' % prob_infarction_aspirin)
print('Infarction probability (placebo): %.4f' % prob_infarction_placebo)
print('Infarction probability decrease: %.4f' % (prob_infarction_placebo - prob_infarction_aspirin))

Infarction probability (aspirin): 0.0094
Infarction probability (placebo): 0.0171
Infarction probability decrease: 0.0077


In [30]:
def proportions_confint_diff_ind(sample1, sample2, alpha=0.05):    
    z = stats.norm.ppf(1 - alpha / 2.)   
    p1 = float(sum(sample1)) / len(sample1)
    p2 = float(sum(sample2)) / len(sample2)
    
    left_boundary = (p1 - p2) - z * np.sqrt(p1 * (1 - p1)/ len(sample1) + p2 * (1 - p2)/ len(sample2))
    right_boundary = (p1 - p2) + z * np.sqrt(p1 * (1 - p1)/ len(sample1) + p2 * (1 - p2)/ len(sample2))
    
    return (left_boundary, right_boundary)

In [31]:
print('Confidence interval: [%.4f, %.4f]' % proportions_confint_diff_ind(placebo_data, aspirin_data))


Confidence interval: [0.0047, 0.0107]


In [32]:
def odds(data):
    p = data.sum() / data.shape[0]
    return p / (1 - p)

In [33]:
odds_aspirin = odds(aspirin_data)
print('Odds aspirin: %.4f' % odds_aspirin)
odds_placebo = odds(placebo_data)
print('Odds aspirin: %.4f' % odds_placebo)

Odds aspirin: 0.0095
Odds aspirin: 0.0174


In [34]:
print('Times decrease of infarction: %.4f' % (odds_placebo / odds_aspirin))


Times decrease of infarction: 1.8321


In [35]:
def get_bootstrap_samples(data, n_samples):
    indices = np.random.randint(0, len(data), (n_samples, len(data)))
    samples = data[indices]
    return samples

In [36]:
def stat_intervals(stat, alpha):
    boundaries = np.percentile(stat, [100 * alpha / 2., 100 * (1 - alpha / 2.)])
    return boundaries

In [47]:
np.random.seed(0)
odds_aspirin_data = list(map(odds, get_bootstrap_samples(aspirin_data, 1000)))
odds_placebo_data = list(map(odds, get_bootstrap_samples(placebo_data, 1000)))

In [51]:
delta = list(map(lambda x: x[1] / x[0], zip(odds_aspirin_data, odds_placebo_data)))

In [52]:
print("95% confidence interval for the difference",  stat_intervals(delta, 0.05))

95% confidence interval for the difference [1.44419465 2.34321168]
